In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import statsmodels.stats.api as sms
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.tsaplots import plot_acf
import warnings
import csv
import json
import string
import re

In [3]:
# required names, change the first 2 lines
year_range = '2024-2025'
json_num = '3'
# files must already exist, auto adjusts based on previous 2 lines
exportify_data = 'cols'+year_range+'_Exportify.csv'
json_og_data = 'Streaming_History_Audio_'+year_range+'_'+json_num+'.json'
# creates new files, used for analysis later
og_data = year_range+'_og.csv'
soundiz_format = 'soundiz_'+year_range+'.csv'
with_play_count = 'Combined_Play_Count'+year_range+'.csv'

In [4]:
df_exportify = pd.read_csv(exportify_data)
df_exportify = df_exportify.drop(["Track ID", "Added By", "Record Label", "Energy", "Mode"], axis = 1)

In [5]:
with open(json_og_data, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Convert to DataFrame
df_og = pd.DataFrame(data)
df_og.to_csv(og_data, index=False)
df_title_artist_album = df_og[["ts", "master_metadata_track_name", "master_metadata_album_artist_name", "master_metadata_album_album_name"]]
df_title_artist_album.to_csv(soundiz_format, index=False)

In [6]:
freq_count = df_title_artist_album.groupby('master_metadata_track_name', sort = False).count().drop(["master_metadata_album_artist_name", "master_metadata_album_album_name"], axis=1)
freq_count

,ts
master_metadata_track_name,
Rainy Days,254
Blue,214
Love Me Again,230
Slow Dancing,210
For Us,198
...,...
Breaking Patterns,1
A Different Kind Of Love,1
Cocopops,1


In [7]:
descriptors = ['- single mix', '- remix', '- live', '- extended', '- radio edit', '- mono']

def normalize(name):
    return name.translate(str.maketrans('', '', string.punctuation)).strip().lower()
    
def trim_descriptors(name):
    name = re.sub(r'\s*-\s*feat.*$', '', name, flags=re.IGNORECASE).strip()
    name = re.sub(r'[$$$\<].*?[$$$\>].*$', '', name, flags=re.IGNORECASE).strip()
    for descriptor in descriptors:
        name = name.lower().replace(descriptor, '').strip()
    return name

In [8]:
freq = []
for i in range(df_exportify.shape[0]):
    norm_exportify = normalize(trim_descriptors(df_exportify.loc[i]['Track Name']))
    found = False
    for j in range(0,freq_count.shape[0]):
        norm_freq_count = normalize(trim_descriptors(freq_count.iloc[j].name))
        if (norm_freq_count == norm_exportify):
            freq.append(freq_count.iloc[j].values[0])
            found = True
            break
    
    if (found == False):
        freq.append(None)
        #print(f"No match found for: {df_exportify.loc[i]['Track Name']} (normalized: {norm_exportify})")

In [9]:
a = pd.Series(freq, name='Play Count')
df_play_count = pd.concat([df_exportify, a], axis=1)
df_new = df_play_count.dropna(subset=['Play Count'])

In [10]:
df_new.to_csv(with_play_count, index=False)